In [256]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [257]:
index_name = "course-questions-hw"

In [258]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

# Delete the index if it exists
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
    print(f"Deleted index: {index_name}")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)


Deleted index: course-questions-hw


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-hw'})

In [259]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [260]:
q = "How to copy a file to a Docker container?"

In [261]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name , body=search_query)   

    result_docs = []

    for hit in response['hits']['hits']:
        doc = hit['_source']
        doc['_score'] = hit['_score']  # Add the score to the document
        result_docs.append(doc)

    return result_docs

In [262]:
def build_prompt(question):
    text = elastic_search(question)

    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context = context_template.format(question=question, text=text)

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    prompt = prompt_template.format(question=question, context=context)

    return prompt

In [263]:
prompt = build_prompt(q)

len(prompt)

2070

In [264]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How to copy a file to a Docker container?

    CONTEXT:
    Q: How to copy a file to a Docker container?
    A: [{'text': "You can use round() function or f-strings\nround(number, 4)  - this will round number up to 4 decimal places\nprint(f'Average mark for the Homework is {avg:.3f}') - using F string\nAlso there is pandas.Series. round idf you need to round values in the whole Series\nPlease check the documentation\nhttps://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.round.html#pandas.Series.round\nAdded by Olga Rudakova", 'section': '2. Machine Learning for Regression', 'question': 'How to output only a certain number of decimal places', 'course': 'machine-learning-zoomcamp', '_score': 39.239384}, {'text': 'Type the following command:\ngit config -l | grep url\nThe output should loo

In [265]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")   

# Count tokens
tokens = encoding.encode(prompt)
num_tokens = len(tokens)
num_tokens

494